Может читать статьи и получать чистый текст и все ссылки в этих статьях. Тексты ссылок могут пригодиться. 

In [13]:
from wiki_dump_reader import Cleaner, iterate

cleaner = Cleaner()
i = 0
test = open('textwiki.txt', 'w', encoding='utf8')
for title, text in iterate(r'F:\CODE\Projects\work\huawei-wiki\dumps\ru\ruwiki-20220701-pages-articles-multistream1.xml-p1p224167'):
    if i > 10:
        break
    text = cleaner.clean_text(text)
    cleaned_text, links = cleaner.build_links(text)
    i += 1
    print(f'text: {cleaned_text}\n___\nLinks: {links}', file=test)
test.close()

Попытка номер два с саксом

In [8]:
path = r'F:\CODE\Projects\work\huawei-wiki\dumps\ru\ruwiki-20220701-pages-articles-multistream6.xml-p9585766p9787309'

In [34]:
import xml.sax

class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))

In [35]:
# Object for handling xml
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

for i, line in enumerate(open(path, encoding='utf8')):
    parser.feed(line)
    
    # Stop when 3 articles have been found
    if len(handler._pages) > 100:
        break
        
print([x[0] for x in handler._pages], sep='\n\n')

['Ғәбиҙулла Ғәбделхәким улы Ҡорбанғәлиев', 'Мөхәммәт-Ғәбделхәй Ҡорбанғәлиев', 'Мөхетдин Хафиз улы Корбангалиев', 'Rahym Muhamedowiç Kurbanmämmedow', 'Täçmämmet Gurbanmämmedow', 'Ýolly Gurbanmyradow', 'Gurbanmyrat Gurbanmyradow', 'Kerim Gurbannepesow', 'Ağadadaş Qurbanov', 'Abdulla Abdurahmonovich Qurbonov', 'Anvar Sarmanovich Qurbonov', 'Afad Məhəmməd oğlu Qurbanov', 'Əhməd Qurbanov', 'Ahmadjon Qurbonov', 'Bahodir Nizomovich Qurbonov', 'Vaqif Qurbanov', 'Habil Surxay oğlu Qurbanov', 'Hamlet Qurbanov', 'İbrahim Musa oğlu Qurbanov', 'Qurban Səid oğlu Qurbanov', 'Black Lion', 'Maqsud Rüstəm oğlu Qurbanov', 'Мамадали Қурбонович Қурбонов', 'Mamadali Qurbonovich Qurbonov', 'Merdan Gurbangeldiýewiç Gurbanow', 'Олим Қурбонов', 'Ravshanbek Davletovich Qurbonov', 'Рашад Афат оглы Курбанов', 'Ruslan Qurbonov', 'Ruslan Eldaroviç Qurbanov', 'Sənan Qurbanov', 'Sahetmyrat Gurbanow', 'Убайдулло Абдулло Қурбон', "O'tkir Qurbonov", 'Şıxəli Qurban oğlu Qurbanov', 'Şöhrat Kurbanow', 'Юсуп Гурбанов', 'Arzu

In [36]:
handler._pages[0]

('Ғәбиҙулла Ғәбделхәким улы Ҡорбанғәлиев',
 '#REDIRECT [[Курбангалиев, Габидулла Габдул-Хакимович]]')

In [ ]:
for page in handler._pages:
    if page[1].startswith('#REDIRECT'):
        print(page[1])

In [27]:
import re

for page in handler._pages:
    s = re.search(r'\[\[Категория:(.*?)]]', page[1])
    if s:
        print(s.group(1))

Населённые пункты Заволжского района (Ивановская область)


In [38]:
import mwparserfromhell 

print(handler._pages[1][0])

# Create the wiki article
wiki = mwparserfromhell.parse(handler._pages[1][1])

Мөхәммәт-Ғәбделхәй Ҡорбанғәлиев


In [39]:
wikilinks = [x.title for x in wiki.filter_wikilinks()]
print(f'There are {len(wikilinks)} wikilinks.')
wikilinks[:5]

There are 1 wikilinks.


['Курбангалиев, Мухаммед-Габдулхай']

Темплейты

In [40]:
templates = wiki.filter_templates()
print(f'There are {len(templates)} templates.')
for template in templates:
    print(template.name)

There are 0 templates.


In [ ]:
# search template
infobox = wiki.filter_templates(matches = 'Infobox radio station')[0]
infobox